# Com utilitzar això

## Canviar les variables
`comer = 'LERSA'`
`PATH = '/home/puig/Fitxers de Medidas/CNMC_TOTALYEAR/{comer}/'.format(comer=comer)`

# Crear els següents directoris dins CNMC_TOTALYEAR
- CLMAGS
- CLMAG5AS
- F1S

#### Nota: Revisar tarifes 3.1 amb més de 30000V de tensió. Hauríen de portar el codi EB i no el tenim a l'agree_tensio, així que s'hauria de revisar. Si no en tenim cap, les 3.1 van a la columna de 1kV a 30kV. Revisar el mateix per les tarifes 6.1


# Validador Hores

In [1]:
# Validador
def val(n_hours, df, ):
    for tarifa in ('21', '2A'):
        if len(df[(df.agree_tarifa == tarifa) & (df.agree_dh == 'E1')]) != n_hours:
            print('Tarifa no te totes les hores: {}'.format(tarifa))
        if len(df[(df.agree_tarifa == tarifa) & (df.agree_dh == 'E2')]) != n_hours:
            print('Tarifa no te totes les hores: {}'.format(tarifa))
    print 'Ha acabat validacio'

def val_clmag5a(n_hours, df_tmp):
    tarifa = list(set(df_tmp.agree_tarifa))[0]
    if tarifa == '2.0 DHS':
        msg = ('Ha acabat validacio {}'.format(tarifa))
        print msg
        return True
    if len(df_tmp) != n_hours:
        msg = ('Tarifa {} no te totes les hores df: {} vs {}!!!!'.format(tarifa, len(df_tmp), n_hours)) 
        print msg
        return False
    else:
        msg = ('Ha acabat validacio {}'.format(tarifa))
        print msg
        return True

def val_f1(n_hours, df_tmp):
    if len(df_tmp) != n_hours:
        msg = ('Tarifa 61 no te totes les hores df: {} vs {}!!!!'.format(len(df_tmp), n_hours)) 
        print msg
        return False
    else:
        msg = ('Ha acabat validacio {}'.format('61'))
        print msg
        return True
    
def val_clmag(n_hours, df):
    for tarifa in ('3.0', ):
        if len(df[(df.agree_tarifa == tarifa) & (df.dh == 'E3')]) != n_hours:
            print('Tarifa no te totes les hores: {}'.format(tarifa))
    print 'Ha acabat validacio'

## Netejar directori F1

In [2]:
def extract_only_last_version(file_type=None):
    if not file_type:
        file_type = 'F1'
    if file_type == 'F1':
        #Clean F1
        base_path = '{}{}S/'.format(PATH, file_type)
        data_files = []
        for _file in os.listdir(base_path):
            data_files.append({'filename': _file, 'sub_filename': _file[:-11], 'generation_date': _file[17:-2], 'version': _file[-1:]})
        df_f1s = pd.DataFrame(data_files)
        df_check = df_f1s.groupby(['sub_filename']).aggregate({'version': 'count'}).reset_index()
        for dup in list(df_check[df_check['version'] != 1]['sub_filename']):
            df_x = df_f1s[df_f1s.sub_filename == dup].sort_values(['generation_date', 'version'])
            df_x = df_x.drop(df_x.index[len(df_x)-1])
            for filename in list(df_x['filename']):
                print('{}'.format(filename))
                os.remove('{}{}'.format(base_path, filename))
    elif file_type == 'CLMAG5A':
        #Clean CLMAG5A
        pass
    elif file_type == 'CLMAG':
        #Clean CLMAG
        pass

In [13]:
def check_totally_f1():
    base_path = '{}{}S/'.format(PATH, 'F1')
    data_files = []
    for _file in os.listdir(base_path):
        data_files.append({'filename': _file, 'sub_filename': _file[:-11], 'month': _file[8:-13], 'generation_date': _file[17:-2], 'version': _file[-1:]})
    df_f1s = pd.DataFrame(data_files)
    return df_f1s.groupby(['month']).aggregate({'version': 'count'})
total_days = check_totally_f1()['version'].sum()
print('Total dies: {}'.format(total_days))
check_totally_f1()

Total dies: 365


,version
month,
201801,31
201802,28
201803,31
201804,30
201805,31
201806,30
201807,31
201808,31
201809,30


## Definició imports i funcions

In [3]:
from measuring_tools.parsers import parser
from measuring_tools.skeleton_files import cols
from measuring_tools.comparative_tools import check_clmag5as, totalizer
import os
import pandas as pd
import numpy as np

comer = 'BINEFAR'
PATH = '/home/puig/Fitxers de Medidas/CNMC_TOTALYEAR/{comer}/'.format(comer=comer)

## Escriure fitxers

In [4]:
def write_excel(data_f, name_sheet):
    from openpyxl import load_workbook
    filename = '{}result/{}.xlsx'.format(PATH, 'TOTAL')
    try:
        book = load_workbook(filename)
    except Exception:
        writer = pd.ExcelWriter(filename, engine='openpyxl')
        data_f.to_excel(writer, name_sheet, index=False, columns=['agree_tarifa', 'timestamp', 'measure'])
        writer.save()
        return True
        
    writer = pd.ExcelWriter(filename, engine='openpyxl') 
    writer.book = book
    writer.sheets = dict((ws.title, ws) for ws in book.worksheets)
    data_f.to_excel(writer, name_sheet, index=False, columns=['agree_tarifa', 'timestamp', 'measure'])

    writer.save()
    return True

# CLMAG5A

In [100]:
# Llegim els CLMAG5AS
df = pd.DataFrame(data={})
base_path = '{}{}/'.format(PATH, 'CLMAG5AS')
if len(os.listdir(base_path)) != 12:
    print('FALTEN MESOS!')
print('IMPORTANT DADES CLMAG5A')

for _file in os.listdir(base_path):
    path_clmag5a = base_path + _file
    print('Reading... {}'.format(path_clmag5a))
    df_tmp = parser.read_CLMAG5A(path_clmag5a)
    if df.empty:
        df = df_tmp
    else:
        df = df.append(df_tmp)

# AGRUPEM PER TARIFA
df_2tariff = df.groupby(['agree_tarifa', 'agree_dh', 'timestamp', 'estacio']).aggregate({'measure': 'sum'}).reset_index()
df_2tariff.sort_values(['timestamp', 'agree_tarifa'])

# Escrivim els fitxers, un per tarifa
df_2tariff = df_2tariff.replace('21', '2.1')
df_2tariff = df_2tariff.replace('2A', '2.0')
df_2tariff['agree_tarifa'] = np.where(
    (df_2tariff['agree_dh'] == 'E2'), df_2tariff['agree_tarifa'] + ' DHA', df_2tariff['agree_tarifa']
)
df_2tariff['agree_tarifa'] = np.where(
    (df_2tariff['agree_dh'] == 'E3'), df_2tariff['agree_tarifa'] + ' DHS', df_2tariff['agree_tarifa']
)
df_2tariff['agree_tarifa'] = np.where(
    (df_2tariff['agree_dh'] == 'E1'), df_2tariff['agree_tarifa'] + ' A', df_2tariff['agree_tarifa']
)

tariffs = list(set(df_2tariff.agree_tarifa))
for tariff in tariffs:
    df_tariff = df_2tariff[df_2tariff.agree_tarifa == tariff]
    if val_clmag5a(8760, df_tariff):
        write_excel(df_tariff, str(tariff))
        df_tariff.to_csv('{}result/{}.csv'.format(PATH, tariff), columns=['timestamp', 'measure', 'agree_tarifa'], sep=';', index=None)

IMPORTANT DADES CLMAG5A
Reading... /home/puig/Fitxers de Medidas/CNMC_TOTALYEAR/BINEFAR/CLMAG5AS/CLMAG5A_0291_201804_20180611.1
Reading... /home/puig/Fitxers de Medidas/CNMC_TOTALYEAR/BINEFAR/CLMAG5AS/CLMAG5A_0291_201801_20180412.3
Reading... /home/puig/Fitxers de Medidas/CNMC_TOTALYEAR/BINEFAR/CLMAG5AS/CLMAG5A_0291_201809_20181207.1
Reading... /home/puig/Fitxers de Medidas/CNMC_TOTALYEAR/BINEFAR/CLMAG5AS/CLMAG5A_0291_201812_20190222.1
Reading... /home/puig/Fitxers de Medidas/CNMC_TOTALYEAR/BINEFAR/CLMAG5AS/CLMAG5A_0291_201811_20190118.1
Reading... /home/puig/Fitxers de Medidas/CNMC_TOTALYEAR/BINEFAR/CLMAG5AS/CLMAG5A_0291_201807_20180925.1
Reading... /home/puig/Fitxers de Medidas/CNMC_TOTALYEAR/BINEFAR/CLMAG5AS/CLMAG5A_0291_201810_20181212.1
Reading... /home/puig/Fitxers de Medidas/CNMC_TOTALYEAR/BINEFAR/CLMAG5AS/CLMAG5A_0291_201808_20181108.1
Reading... /home/puig/Fitxers de Medidas/CNMC_TOTALYEAR/BINEFAR/CLMAG5AS/CLMAG5A_0291_201803_20180510.1
Reading... /home/puig/Fitxers de Medidas

# CLMAG

In [99]:
def read_CLMAG(path):
    #positions = [x for x in xrange(11, 107, 4)]
    positions = [x for x in xrange(11, 110, 4)]
    hours_map = {
        11: '01', 15: '02', 19: '03', 23: '04', 27: '05', 31: '06', 35: '07', 39: '08', 43: '09', 47: '10', 
        51: '11', 55: '12', 59: '13', 63: '14', 67: '15', 71: '16', 75: '17', 79: '18', 83: '19', 87: '20', 
        91: '21', 95: '22', 99: '23', 103: '24', 107: '25'
    }
    R1_code = ';R1;'
    consumptions_clmag = []

    lines = []
    with open(path, 'r') as filename:
        for line in filename:
            # zfills month / distri / comer / type
            a = line.split(';')
            a[1] = a[1].zfill(2)
            a[3] = a[3].zfill(4)
            a[4] = a[4].zfill(4)
            a[9] = a[9].zfill(2)
            lines.append(str(';'.join(a)))
    clmag_aggs = [agg[11:35] for agg in lines]
    df_aggs = pd.DataFrame(data=clmag_aggs).drop_duplicates()
    clmag_aggs = [x[0] for x in df_aggs.values.tolist()]
    aggregation_dict = dict.fromkeys(cols.CLMAG_COLS)
    report_map = {}
    for x in lines:
        if R1_code not in x:
            # Get aggregation and date
            timestamp_base = '/'.join([x[6:10], x[3:5], x[:2]])
            consumption_tmp = x.split(';')
            aggregation = x[11:35]
            
            # Initialize set
            if aggregation not in report_map:
                report_map[aggregation] = []

            # Set timestamp-measure
            for pos in positions:
                timestamp = '{} {}:00'.format(timestamp_base, hours_map[pos])
                if timestamp.find(" 25:00") != -1 and timestamp != "2018/10/28 25:00":
                    continue
                report_map[aggregation].append({'timestamp': timestamp, 'measure': int(consumption_tmp[pos])})
    return report_map


# Llegim els CLMAGS
df = pd.DataFrame(data={})
base_path = '{}{}/'.format(PATH, 'CLMAGS')
for _file in os.listdir(base_path):
    path_clmag = base_path + _file
    print('Reading... {}'.format(path_clmag))
    
    rp = read_CLMAG(path_clmag)
    data = []
    for k, v in rp.iteritems():
        aggregation = dict(zip(['distri', 'comer', 'provincia', 'tensio', 'agree_tarifa', 'dh', 'tipo'], tuple(k.split(';'))))
        for s in v:
            _agg = aggregation.copy()
            _agg['timestamp'] = s['timestamp']
            _agg['measure'] = s['measure']
            data.append(_agg)
    df_tmp = pd.DataFrame(data=data)
    if df.empty:
        df = df_tmp
    else:
        df = df.append(df_tmp)
# Vestim el dataframe
for key in ('tipo', 'distribuidora'):
    df['tipo'] = df['tipo'].apply(lambda x: x.zfill(2))
df_2tariff = df.groupby(['agree_tarifa', 'dh', 'timestamp']).aggregate({'measure': 'sum'}).reset_index()
df_2tariff.sort_values(['timestamp', 'agree_tarifa'])

# Drop hour change march hour
df_2tariff = df_2tariff.drop(df_2tariff[df_2tariff.timestamp == '2018/03/25 03:00'].index)
df_2tariff = df_2tariff.sort_values(['timestamp'])

# Escrivim els fitxers, un per tarifa
df_2tariff = df_2tariff.replace('30', '3.0')
df_2tariff = df_2tariff.replace('31', '3.1')
df_2tariff['agree_tarifa'] = np.where(
    ((df_2tariff['agree_tarifa'] == '3.1') & (df_2tariff['dh'] == 'EB')), 
    df_2tariff['agree_tarifa'] + ' 30 a 36kV', 
    df_2tariff['agree_tarifa']
)

tariffs = list(set(df_2tariff.agree_tarifa))
for tariff in tariffs:
    df_tariff = df_2tariff[df_2tariff.agree_tarifa == tariff]
    # Write file
    df_tariff.to_csv('{}result/{}.csv'.format(PATH, tariff), columns=['timestamp', 'measure'], sep=';', index=None)
    val_clmag(8760, df_tariff)
    write_excel(df_tariff, tariff)

Reading... /home/puig/Fitxers de Medidas/CNMC_TOTALYEAR/BINEFAR/CLMAGS/CLMAG_0291_201801_20180323.1
Reading... /home/puig/Fitxers de Medidas/CNMC_TOTALYEAR/BINEFAR/CLMAGS/CLMAG_0291_201812_20190222.1
Reading... /home/puig/Fitxers de Medidas/CNMC_TOTALYEAR/BINEFAR/CLMAGS/CLMAG_0291_201808_20181108.1
Reading... /home/puig/Fitxers de Medidas/CNMC_TOTALYEAR/BINEFAR/CLMAGS/CLMAG_0291_201809_20181207.1
Reading... /home/puig/Fitxers de Medidas/CNMC_TOTALYEAR/BINEFAR/CLMAGS/CLMAG_0291_201811_20190118.1
Reading... /home/puig/Fitxers de Medidas/CNMC_TOTALYEAR/BINEFAR/CLMAGS/CLMAG_0291_201803_20180510.1
Reading... /home/puig/Fitxers de Medidas/CNMC_TOTALYEAR/BINEFAR/CLMAGS/CLMAG_0291_201806_20180920.1
Reading... /home/puig/Fitxers de Medidas/CNMC_TOTALYEAR/BINEFAR/CLMAGS/CLMAG_0291_201805_20180713.1
Reading... /home/puig/Fitxers de Medidas/CNMC_TOTALYEAR/BINEFAR/CLMAGS/CLMAG_0291_201804_20180611.1
Reading... /home/puig/Fitxers de Medidas/CNMC_TOTALYEAR/BINEFAR/CLMAGS/CLMAG_0291_201802_20180417.1


# F1

In [14]:
df = pd.DataFrame(data={})
base_path = '{}{}/'.format(PATH, 'F1S')
for _file in os.listdir(base_path):
    path_f1 = base_path + _file
    df_tmp = parser.read_F1(path_f1)
    if df.empty:
        df = df_tmp
    else:
        df = df.append(df_tmp)
        
df_2tariff = df.groupby(['timestamp', 'estacio']).aggregate({'measure': 'sum'}).reset_index()
df_2tariff['agree_tarifa'] = '61'
df_2tariff = df_2tariff.sort_values(['timestamp'])
if val_f1(8760, df_2tariff):
    df_2tariff.to_csv('{}result/{}.csv'.format(PATH, '61'), columns=['timestamp', 'measure'], sep=';', index=None)
write_excel(df_2tariff, '61')

Ha acabat validacio 61


True

## Our data validations

In [108]:
extract_only_last_version()

F1_0291_20180813_20180814.0
F1_0291_20180818_20180819.0
F1_0291_20180924_20180925.0
F1_0291_20181004_20181005.0
F1_0291_20181024_20181029.1
F1_0291_20181025_20181029.1
F1_0291_20181026_20181029.1
F1_0291_20181027_20181029.1
F1_0291_20181028_20181029.1
F1_0291_20181028_20181029.2
